## Script permettant de récupérer les Json Orthoinspector et d'obtenir un fichier par protéine humaine avec les identifiant Uniprot des protéines ortologue pour chaque protéine (humaine et primate)
Input: aucun  
Output:  
    Stockage des réponse Json Orthoinspector  
    Stockage du processing des json avec un fichie par espèce contenant les protéines ID  
    Stockage d'un unique tableau contenant en colonne chaque espèce et en ligne chaque protéine ID  
    Stockage d'un fichier par ligne contenant l'ID humain et les ID de chaque primate

In [ ]:
%%bash
# Récupérer les fichiers json qui listent tout les ortholgoues entre humain et espèce d'intérêt 
taxonID=(60711 9541 9544 9555 9595 9598 9601 61853 9483 30611)
name=(C.sabaeus M.fascicularis M.mulatta P.anubis G.gorilla H.sapiens P.troglodytes P.abellii N.leucogenys C.jacchus O.garnettii)
human=9606
for i in ${taxonID[@]}
do
    curl -X GET "https://lbgi.fr/orthoinspectorv3/api/Eukaryota/species/$human/orthologs/$i" -H  "accept: application/json" > ../raw/orthoinspector_json/$i.json
done

In [ ]:
# Traiter les fichiers json pour avoir un colonne prot humaine et une colonne orthologue chez espèce d'intérêt
from os import listdir
from os.path import isfile, join
import json


taxonID = [60711, 9541, 9544, 9555, 9595, 9598, 9601, 61853, 9483, 30611]
human = 9606

myfiles = listdir("../raw/orthoinspector_json/")
for i in range(10):
    f = open("../raw/orthoinspector_json-processed/"+str(taxonID[i])+".txt", "w")
    with open("../raw/orthoinspector_json/"+str(taxonID[i])+".json") as json_file:
        data = json.load(json_file)
        f.write("9606\t"+str(taxonID[i])+"\n")
        for human_prot in data:
            if human_prot == data[human_prot]['orthologs'][0]:
                f.write(human_prot+"\t"+ str("NaN"+"\n"))
            else:
                f.write(human_prot+"\t"+ str(data[human_prot]['orthologs'][0]+"\n"))
            #f.write((human_prot+"\t"+ ' '.join([i for i in data[human_prot]['orthologs']])+"\n"))
    f.close()

Species
	
Taxon
	
Model
	
NCBI Taxonomy id
Chlorocebus sabaeus 	Metazoa 		60711
Macaca fascicularis 	Metazoa 		9541
Macaca mulatta 	Metazoa 		9544
Papio anubis 	Metazoa 		9555
Gorilla gorilla gorilla 	Metazoa 		9595
Homo sapiens 	Metazoa 		9606
Pan troglodytes 	Metazoa 		9598
Pongo abelii 	Metazoa 		9601
Nomascus leucogenys 	Metazoa 		61853
Callithrix jacchus 	Metazoa 		9483
Otolemur garnettii 	Metazoa 		30611

In [ ]:
# Fusion des fichier traiter précédent pour passer d'un fichier par comparaison à un fichier pour toute les comparaison
import pandas as pd
from os import listdir
from os.path import isfile, join
import json

myfiles2 = listdir("../raw/orthoinspector_json-processed/")
main_df = pd.read_csv('../raw/orthoinspector_json-processed/'+myfiles2[0], sep='\t', header=0)
for i in range(1, 10):
    df_temp = pd.read_csv('../raw/orthoinspector_json-processed/'+myfiles2[i], sep='\t', header=0)
    main_df = main_df.merge(df_temp, on="9606", how="outer")
gene_names = main_df.copy()
gene_names

In [ ]:
gene_names.to_csv("../raw/orthoinspector_json-processed/allProteineName.txt", sep="\t", index=False)

In [ ]:
# Préparation des identifiants pour query de séquence par fichier. Un fichier par comparaison (20 000 au total) à récupérer
# Une séquence par ligne
for i in range(20265):
    f = open("../raw/uniprot-sequence/"+str(gene_names.iloc[i,0])+".id", "w", newline='\n')
    query = gene_names.iloc[i,:].dropna().tolist()
    query = ' '.join(query)
    query = query.split(" ")
    for i in query:
        f.write(str(i)+"\n")
    f.close()